In [ ]:
import pandas as pd
import numpy as np
import os
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import col

#NLP

In [ ]:
NASA = spark.sql("select ARTIKEL_EAN as gtin, GLN as gln from standaardtabellen.nasastandaardtabel")
df = spark.sql('select * from StandaardTabellen.PreprocessedAttributeData')
df = df.drop('GPC', 'commerciele_velden', 'commercieel_ingredienten', 'commercieel_ingredienten_toevoegingen')
df = NASA.join(df, ['gtin', 'gln'], how = 'left')
print("Shape van DF (row, columns) ",(df.count(),len(df.columns))) # Shape of dataframe

df_pandas = df.toPandas()

velden = {'regulated_product_name':"string",'functional_name':"string",'label_description':"string",'trade_item_description':"string",'ingredients':"string",'description_short':"string", 
          'consumer_usage_instruction':"string",'trade_item_feature_benefit':"string",'marketing_message':"string",'preparation_type':"string",'preparation_instruction':"string",'toevoegingen_ingredienten':"string"}

for i in velden:
    df = df.withColumn(i, concat_ws(" ", col(i)))
    
df_pandas = df_pandas.astype(velden) # object to string
df_NLP = df_pandas[['gtin', 'gln', 'regulated_product_name', 'functional_name', 'label_description', 'trade_item_description', 'ingredients', 'description_short', 'consumer_usage_instruction', 
                    'trade_item_feature_benefit', 'marketing_message', 'preparation_type', 'preparation_instruction', 'toevoegingen_ingredienten']].copy()



display(df_NLP)

In [ ]:

# Table 1
spark.sql("use standaardtabellen")
NASA_totaal = spark.sql("select * from nasastandaardtabel")
NASA_totaal = NASA_totaal.select(col("NASA_NR"), col("PRIMARY_EAN").alias("ARTIKEL_EAN"), col("GLN").alias("gln_nasa"), col("SCHAPSTICKER_OMSCHRIJVING"),col("ASSGROEP_OMSCHRIJVING"), col("KASSABON_OMSCHRIJVING"), col("LANGE_OMSCHRIJVING"), col("EENHEID")
                                , col("INHOUD"))
# Table 2
GLNAH = spark.read.format('delta').load('/mnt/Prd_adls/Conformed/TIMS/TradeItem/ahTradeItem/Data/').filter('__DeletedFlag == 0')
GLNAH = GLNAH.select(col("gtin").alias("gtin1"), col("gln").alias('gln1'), col("glnAH"), col("ndDepartment"))
 
# Table 3
spark.read.format('delta').load('/mnt/Prd_adls/Conformed/TIMS/TradeItem/tradeItemInfo/Data/').filter('__DeletedFlag == 0').registerTempTable('tradeItemInfo')
tradeItemInfo = spark.sql("select gtin, gln, coalesce (tradeItemInfo.mvlang_tradeItemDescMod_tradeItemDescInfo_tradeItemDesc['nl'], tradeItemInfo.mvlang_tradeItemDescMod_tradeItemDescInfo_tradeItemDesc['en']) as TradeItemDescr, coalesce(lang_tradeItemDescMod_tradeItemDescInfo_functionalName['nl'], lang_tradeItemDescMod_tradeItemDescInfo_functionalName['en']) as functionalName, coalesce(mvlang_tradeItemDescMod_tradeItemDescInfo_labelDesc['nl'], mvlang_tradeItemDescMod_tradeItemDescInfo_labelDesc['en']) as labelDescription, coalesce(mvlang_tradeItemDescMod_tradeItemDescInfo_regulatedProdName['nl'], mvlang_tradeItemDescMod_tradeItemDescInfo_regulatedProdName['en']) as regulatedProdName, coalesce(mvlang_foodAndBeverageIngredientMod_ingredientStatement['nl'], mvlang_foodAndBeverageIngredientMod_ingredientStatement['en']) as ingredient_informatie from tradeItemInfo")
# coalesce, returned de eerste non-null value in een list
 
 
# Join several Tables
NASA_totaal = NASA_totaal.join(GLNAH, (NASA_totaal.ARTIKEL_EAN == GLNAH.gtin1) & (NASA_totaal.gln_nasa == GLNAH.gln1), how = "left")
NASA_totaal = NASA_totaal.join(tradeItemInfo, (NASA_totaal.ARTIKEL_EAN == tradeItemInfo.gtin) & (NASA_totaal.gln_nasa == tradeItemInfo.gln), how = "left")
print((NASA_totaal.count(),len(NASA_totaal.columns))) 
NASA_totaal = NASA_totaal.filter(NASA_totaal.glnAH == True)
 
# Select Features
NASA = NASA_totaal.select("ARTIKEL_EAN", "gln_nasa","glnAH","regulatedProdName","functionalName", "labelDescription","TradeItemDescr","ingredient_informatie")
NASA = NASA.withColumnRenamed("ARTIKEL_EAN", "gtin") # hernoem naar gtin, zodat we department kunnen mergen.
NASA = NASA.withColumnRenamed("gln_nasa", "gln") # hernoem naar gtin, zodat we department kunnen mergen.
print("Shape van DF (row, columns) ",(NASA.count(),len(NASA.columns))) # Shape of dataframe
 
# Arrays naar strings
NASA = NASA.withColumn("regulatedProdName", concat_ws(" ", "regulatedProdName"))
NASA = NASA.withColumn("functionalName", concat_ws(" ", "functionalName"))
NASA = NASA.withColumn("labelDescription", concat_ws(" ", "labelDescription"))
NASA = NASA.withColumn("TradeItemDescr", concat_ws(" ", "TradeItemDescr"))
NASA = NASA.withColumn("ingredient_informatie", concat_ws(" ", "ingredient_informatie"))

NASA = NASA.filter(NASA.glnAH == True)
df_pandas = NASA.toPandas()

df_pandas = df_pandas.astype({"regulatedProdName":"string", "functionalName":"string","labelDescription":"string","TradeItemDescr":"string","ingredient_informatie":"string"}) # object to string
df_NLP = df_pandas[['gtin','gln','regulatedProdName','functionalName','labelDescription','TradeItemDescr','ingredient_informatie']].copy()

#Controle, voeg extra kolommen toe

In [ ]:
# NASA 
spark.sql("use standaardtabellen")
NASA_CONTROLE = spark.sql("select * from nasastandaardtabel")
NASA_CONTROLE = NASA_CONTROLE.select(col("NASA_NR"), col("PRIMARY_EAN").alias("ARTIKEL_EAN"), col("GLN").alias("gln_nasa"), col("SCHAPSTICKER_OMSCHRIJVING"),col("ASSGROEP_OMSCHRIJVING"))

GLNAH_CONTROLE = spark.read.format('delta').load('/mnt/Prd_adls/Conformed/TIMS/TradeItem/ahTradeItem/Data/').filter('__DeletedFlag == 0')
GLNAH_CONTROLE = GLNAH_CONTROLE.select(col("gtin").alias("gtin1"), col("gln").alias('gln1'), col("glnAH"))

NASA_CONTROLE = NASA_CONTROLE.join(GLNAH_CONTROLE, (NASA_CONTROLE.ARTIKEL_EAN == GLNAH_CONTROLE.gtin1) & (NASA_CONTROLE.gln_nasa == GLNAH_CONTROLE.gln1), how = "left")
NASA_CONTROLE = NASA_CONTROLE.filter(NASA_CONTROLE.glnAH == True)

NASA_CONTROLE = NASA_CONTROLE.select("ARTIKEL_EAN", "gln_nasa","glnAH")
NASA_CONTROLE = NASA_CONTROLE.withColumnRenamed("ARTIKEL_EAN", "gtin") # hernoem naar gtin, zodat we department kunnen mergen.

#Tables: Text- en Numerieke Features. Daarnaast worden er enkel food producten uitgefilterd.

In [ ]:
#basis + nutrienten + food
basis = spark.read.format('delta').load('/mnt/Prd_adls/Conformed/TIMS/TradeItem/ahTradeItem/Data/').filter('__DeletedFlag == 0')
basis = basis.select("gtin", "gln", 'glnAH', 'sgIngredientsRequiredIndicator', 'ndSingleComposed')

food = spark.sql("select gtin, gln, ndFoodNonFood from ndattribuutoutput.ndfoodnonfood")

# NUMERIEKE FEATURES
vezelwijzer = spark.sql("select gtin, gln, Preferred_table, FIBER, Fiber_UOM, NBQ_UOM, SUGAR, SUGAR_UOM, FAT, FASAT, CHOAVL, PROTEINE, SALT, ENER_KJ, ENER_Kcal, Volgorde_Nutrienten from standaardtabellen. Nutrienten") 
vezelwijzer = vezelwijzer.filter(((vezelwijzer.Volgorde_Nutrienten == '0') | 
                                  (vezelwijzer.Volgorde_Nutrienten == '1') | 
                                  (vezelwijzer.Volgorde_Nutrienten == '2') | 
                                  (vezelwijzer.Volgorde_Nutrienten == '3')) & 
                                 (vezelwijzer.Preferred_table == "Ja"))
vezelwijzer = vezelwijzer.drop('Volgorde_Nutrienten', 'Preferred_table')
vezelwijzer = vezelwijzer.drop_duplicates()

df = basis.join(food, ['gtin', 'gln'], how = "left")
df = df.join(vezelwijzer, ['gtin', 'gln'], how = "left")

# TEXT FEATURES, NLP Part

#DF_NLP_spark = spark.sql("select gtin, gln, lemmatized_DUTCH_EN as Lemmatized, padded_seq__EN from default._paddedseq_ingredient")
DF_NLP_spark = spark.sql("select gtin, gln, lemmatized_DUTCH as Lemmatized, padded_seq_ from default._nd_Modelling_NLP")


DF_NLP_spark = DF_NLP_spark.join(df, ['gtin', 'gln'], how = "left")
DF_NLP_spark = DF_NLP_spark.filter((DF_NLP_spark.glnAH == True) & (DF_NLP_spark.ndFoodNonFood == 'Food'))